In [5]:
import pandas as pd
import re
from tqdm import trange,tqdm

# linenumber에 따라서 검수할 수 있게 3000~4000개 기준 
# linenumber 하나하나 기억해서 1500개 기준
class rule_check:
  
  def start(self):
    
    data, count_data = self.get_data()
    
    linenumber_data = count_data['linenumber'].to_list()
    count = int(count_data['count'][0])
    
    perfect_data, rule_error_data, no_fixed_data, error_data, linenumber_list = self.check(data, linenumber_data)
    
    self.save_data(perfect_data, rule_error_data, no_fixed_data, error_data, linenumber_data, linenumber_list, count)

  # 데이터 호출
  def get_data(self):

    # 데이터 경로
    data = pd.read_csv('C:/Users/jayts/Downloads/텍스트 다듬기 Type B - 20 October 2021 10_03_57 AM/type2_input_1.csv')
        
    count_data = pd.read_csv('C:/Users/jayts/OneDrive/문서/카카오톡 받은 파일/count_data.csv')

    return data, count_data

  def check(self, data, linenumber_data):

    # 데이터 검사 개수
    count = 0
    
    # 정상적인 데이터
    perfect_data = []

    # 수정안된 데이터 개수
    no_fixed_count = 0

    # 수정안한 데이터
    no_fixed_data = []

    # 태그외부 단어를 수정한 데이터 개수
    rule_error_count = 0

    # 태그외부 단어를 수정한 데이터
    rule_error_data = []

    # 에러로 표시된 개수 or 에러로 확인된 데이터 개수
    error_count = 0

    # 에러 데이터
    error_data = []

    #temp 데이터, 사용한 linenumber를 해당 변수에 추가함
    linenumber_list = []
    
    for i in range(0,len(data)):

      lineNumber = int(data.loc[i]['lineNumber'])
      
      if self.linenumber_check(lineNumber, linenumber_data):
        continue
      
      linenumber_list.append(lineNumber)
      
      originalText = str(data.loc[i]['originalText']).replace(u'\xa0', u' ')
      transcription = str(data.loc[i]['transcription']).replace(u'\xa0', u' ')
      
      # 태그와 태그 내부까지 동시 삭제 
      originalText_without_asterisk = re.sub('(\<{3}.*?\>{3})','',originalText).replace('<', '').replace('>', '')
      transcription_without_asterisk = re.sub('(\<{3}.*?\>{3})','',transcription).replace('<', '').replace('>', '')
      if transcription_without_asterisk.lower() == 'error':
        
        error_count = error_count + 1
        error_data.append([lineNumber, originalText, transcription])
        pass
      
      # 변경 전, 후 같다면 변경 안한 것
      elif originalText == transcription:

        no_fixed_count = no_fixed_count + 1
        no_fixed_data.append([lineNumber, originalText, transcription])
      
      # 태그와 태그 내부를 제거한 상태에서 문장이 같지 않을 경우
      elif originalText_without_asterisk != transcription_without_asterisk:
        
        # 모든 공백을 제거 후 비교했을 때 같다면 공백을 건드린 데이터, 공백만을 건드렸다면 넘어가기~
        if originalText_without_asterisk.replace(' ', '') == transcription_without_asterisk.replace(' ', ''):
          perfect_data.append([lineNumber, originalText, transcription])
        
        # 모든 공백을 제거 후 비교했을 때 다르다면 단어를 건드림
        else:
          
          rule_error_count = rule_error_count + 1
          rule_error_data.append([lineNumber, originalText, transcription])
          # flag = True
      #     originalText_without_asterisk = originalText_without_asterisk.replace(' ', '')
      #     transcription_without_asterisk = transcription_without_asterisk.replace(' ', '')
      #     if len(originalText_without_asterisk) > len(transcription_without_asterisk):
      #       for wr in range(0, len(transcription_without_asterisk)):
              
      #         if originalText_without_asterisk[wr] != transcription_without_asterisk[wr] and flag:
      #           print(lineNumber)
      #           print(wr)
      #           print('originalText : ' + originalText_without_asterisk[wr])
      #           print('transcription : ' + transcription_without_asterisk[wr])
      #           flag = False
      #     else :
      #       for wr in range(0, len(originalText_without_asterisk)):
              
      #         if originalText_without_asterisk[wr] != transcription_without_asterisk[wr] and flag:
      #           print(lineNumber)
      #           print(wr)
      #           print('originalText : ' + originalText_without_asterisk[wr])
      #           print('transcription : ' + transcription_without_asterisk[wr])
      #           flag = False
      # # 장상적인 데이터
      else:
        perfect_data.append([lineNumber, originalText, transcription])
        
      count = count + 1
    
    self.print_object(count ,rule_error_count, no_fixed_count, error_count)
    
    return perfect_data, rule_error_data, no_fixed_data, error_data, linenumber_list

  def save_data(self, perfect_data, rule_error_data, no_fixed_data, error_data, linenumber_data, linenumber_list, count):
        
    COLUMNS = ['lineNumber','originalText', 'transcription']
    
    dataframe = pd.DataFrame(perfect_data, columns= COLUMNS)
    # 데이터 저장할 경로
    dataframe.to_csv(f'C:/Users/jayts/OneDrive/바탕 화면/Tictag/카스 데이터/KAS 저장 최종/Type23/test/perfect_data_{count}.csv', index=False, encoding='utf-8-sig')
    
    dataframe = pd.DataFrame(rule_error_data, columns = COLUMNS.append('error'))
    dataframe.to_csv(f'C:/Users/jayts/OneDrive/바탕 화면/Tictag/카스 데이터/KAS 저장 최종/Type23/test/rule_error_data_{count}.csv', index=False, encoding='utf-8-sig')
    
    dataframe = pd.DataFrame(no_fixed_data, columns = COLUMNS)
    dataframe.to_csv(f'C:/Users/jayts/OneDrive/바탕 화면/Tictag/카스 데이터/KAS 저장 최종/Type23/test/no_fixed_data_{count}.csv', index=False, encoding='utf-8-sig')
    
    dataframe = pd.DataFrame(error_data, columns = COLUMNS)
    dataframe.to_csv(f'C:/Users/jayts/OneDrive/바탕 화면/Tictag/카스 데이터/KAS 저장 최종/Type23/test/error_data_{count}.csv', index=False, encoding='utf-8-sig')
    
    
    '''
    사용한 linenumber랑 현재 몇 번째로 보내는 데이터인지 자동으로 슥삭..!
    '''
    linenumber_data = linenumber_data + linenumber_list
    
    count = count + 1
    temp_count = [0 for i in range(len(linenumber_data))]
    temp_count[0] = count
    
    count = temp_count
    
    count_data = [[linenumber_data[i], count[i]] for i in range(0,len(linenumber_data))]
    
    # print(count_data)
    
    dataframe = pd.DataFrame(count_data, columns=['linenumber','count'])
    dataframe.to_csv('Type2/Data/Type_B_output_data/count_data.csv', index=False,)

  def print_object(self, count, rule_error_count, no_fixed_count, error_count):
    
    print(f"검사한 전체 데이터 개수는 {count}개 입니다.")
    print(f"규칙 위반 문장은 {rule_error_count}개 입니다.")
    print(f"변경하지 않은 문장은 {no_fixed_count}개 입니다.")
    print(f"에러 문장은 {error_count}개 입니다.")
    
  def linenumber_check(self, linenumber, linenumber_data):
    
    flag = False
    
    for ln in linenumber_data:
      
      if ln == linenumber:
        
        flag = True
        break
      
     
    
    return flag

In [6]:
a = rule_check()

In [7]:
a.start()

검사한 전체 데이터 개수는 3376개 입니다.
규칙 위반 문장은 44개 입니다.
변경하지 않은 문장은 250개 입니다.
에러 문장은 40개 입니다.


ValueError: 4 columns passed, passed data had 3 columns